# IMPORT

In [762]:
import os, shutil #Controlling Files and paths and folder control
from shutil import copyfile
from pdf2image import convert_from_path #Libary convert pdf file to img file
from google.oauth2 import service_account #Control API Keys
from google.cloud import vision # Vision API from Google
import io
import string, random
import pandas as pd
# Imports the Google Cloud client library
from google.cloud import language_v1
from google.cloud.language_v1 import enums

In [763]:
#from __future__ import unicode_literals, print_function
#from spacy.lang.en import English # updated

# PDF TO TEXT

In [764]:
#Paths that have all folders which use everything
pdfIMGPopplerPath = '/Users/kunal/Documents/VdartResumeProject/Poppler/poppler-0.68.0_x86/poppler-0.68.0/bin/'
imgTxtVisionAPIPath = "/Users/kunal/Documents/VdartResumeProject/APIKEYSGOOGLE/resumeMatcher-pdf2img.json"
runningDocumentPath = '/Users/kunal/Documents/VdartResumeProject/runningDoc/'
sourceFolderResumesPath = '/Users/kunal/Documents/VdartResumeProject/50_resumes/'
excelFilesPath = '/Users/kunal/Documents/VdartResumeProject/ExcelFiles/'
nlpAutoAPIPath = "/Users/kunal/Documents/VdartResumeProject/APIKEYSGOOGLE/resumeMatcher-NLP_create_data.json"
jsonFolderPath = '/Users/kunal/Documents/VdartResumeProject/JSONLFILES/'

In [765]:
def convert_pdf_2_image(uploaded_image_path, uploaded_image):
    #Using the convert_from_path function
    #Same name as pdf but converted to img
    #Watch out for poppler -- necceasary to function
    os.chdir(uploaded_image_path) # Change the working diretory to path that contains the PDF file
    file_name = str(uploaded_image).replace('.pdf','') # file name for png still going to get changed later
    pages = convert_from_path(uploaded_image, 200,poppler_path=pdfIMGPopplerPath) #function to change pdf to img
    pageNumCount = 1 #numbering for all the different images if pdf is multiple pages
    outputNames = []
    for page in pages:
        output_file = file_name+"_"+str(pageNumCount) + '.jpg'#uptaded name for image
        page.save(output_file, 'JPEG')#save img
        pageNumCount +=1
        outputNames.append(output_file)
    return outputNames #names of img

In [766]:
def convert_img_2_text(imagePath):
    #Using API from Google
    #Returns a JSON file but text is extracted from it
    keyDIR = imgTxtVisionAPIPath #JSON key file to call the api
    credentials = service_account.Credentials.from_service_account_file(keyDIR) #using service account to go through google
    client = vision.ImageAnnotatorClient(credentials=credentials) # client api
    with io.open(imagePath, 'rb') as image_file: #opening and reading img
        content = image_file.read() 
    image = vision.types.Image(content=content) # calling and running the client Visison API
    response = client.text_detection(image=image) # JSON return with all values
    texts = response.text_annotations # Go through Json and extract the text detected
    totalString = ''
    for text in texts:
        totalString+=text.description
    totalString = totalString.rsplit(' ', 1)[0] # throws away all the metadata with location -- could be used later
    return totalString #TEXT detected

In [767]:
def deleteIMG(path):
    #Deletes all the images from a folder
    for i in os.listdir(path):
        if i.endswith(".jpg"):
            os.remove(folder_pdf + i)
def deleteEverythingInFolder(folder_pdf):
    #deletes everything in the folder including folders and files
    for file in os.listdir(folder_pdf):
        try:
            shutil.rmtree(folder_pdf+ file) #remove folder
        except NotADirectoryError:
            try:
                os.remove(folder_pdf+ file) # if it is not a folder than it is a file so it removes it also
            except:
                pass
def moveRenameAllFiles(sourceFolderResumes, folder_pdf):
    #Moves all files from source and renames them starting from 100
    #If start from 1 then the ordering will be off
    for realFile in os.listdir(sourceFolderResumes):
        copyfile(sourceFolderResumes + realFile, folder_pdf + realFile) #duplicates all the files from source
    os.chdir(folder_pdf) # move them to the running folder
    documentcounter = 100 # names "Dociment_[num].pdf" start from 100 cause that is how numbering is listed correctly
    for i in os.listdir(folder_pdf):
        fileNameCreator = "Document_" + str(documentcounter) + ".pdf"
        print(i + "   changed to  " + fileNameCreator)
        os.rename(i, fileNameCreator)  # change name of file
        documentcounter+=1

In [768]:
folder_pdf = runningDocumentPath # set var
sourceFolderResumes = sourceFolderResumesPath

In [769]:
deleteEverythingInFolder(folder_pdf)
moveRenameAllFiles(sourceFolderResumes, folder_pdf)
numberOfFiles = len([name for name in os.listdir(folder_pdf) if os.path.isfile(name)]) 
#PNGFilesExist = checkifPNGExists(folder_pdf)
#deletes everything in the folder, copy and moves all files and renames them starting from 100

Document_402.pdf   changed to  Document_100.pdf
Document_403.pdf   changed to  Document_101.pdf
Document_405.pdf   changed to  Document_102.pdf
Document_406.pdf   changed to  Document_103.pdf
Document_407.pdf   changed to  Document_104.pdf
Document_408.pdf   changed to  Document_105.pdf
Document_409.pdf   changed to  Document_106.pdf
Document_410.pdf   changed to  Document_107.pdf
Document_411.pdf   changed to  Document_108.pdf
Document_412.pdf   changed to  Document_109.pdf
Document_413.pdf   changed to  Document_110.pdf
Document_414.pdf   changed to  Document_111.pdf
Document_415.pdf   changed to  Document_112.pdf
Document_417.pdf   changed to  Document_113.pdf
Document_418.pdf   changed to  Document_114.pdf
Document_419.pdf   changed to  Document_115.pdf
Document_420.pdf   changed to  Document_116.pdf
Document_421.pdf   changed to  Document_117.pdf
Document_422.pdf   changed to  Document_118.pdf
Document_423.pdf   changed to  Document_119.pdf
Document_424.pdf   changed to  Document_

In [771]:
#Asks the user which document they want to run and if they don't input anything, its a random document
doubleCheckDocumentInputCount = 0
while True:
    documentRun = input("Which Document do you wanna run?")
    if documentRun == "" and doubleCheckDocumentInputCount >= 1:
        documentRun = (random.choice(os.listdir(folder_pdf)))
        print("You choose RANDOM so your document is " + documentRun)
        break
    elif documentRun == "":
        print("This is a double check if you want a random document?")
        doubleCheckDocumentInputCount+=1
    elif documentRun == "BREAK":
        print("Exiting")
        documentRun = "INVALID"
        break
    else:
        documentRun = "Document_" + documentRun + ".pdf"
        print(os.path.isfile(folder_pdf + documentRun))
        if os.path.isfile(folder_pdf + documentRun):
            print("You choose a specific number so your document is " + documentRun)
            break
        else:
            print("Document was invalid. Input >>> BREAK <<< to stop")
            continue

Which Document do you wanna run?115
True
You choose a specific number so your document is Document_115.pdf


In [772]:
#Makes a folder and create images of each page in PDF document
textList = []
numberPagesList = []
for i in os.listdir(folder_pdf): 
    if i == documentRun:# only runs the file that was choosen by user
        os.chdir(folder_pdf) 
        folderNameCreate = folder_pdf+"Fold_"+i[:-4]+"/" 
        print(folderNameCreate)
        os.mkdir(folderNameCreate) #Create folder
        shutil.move(i,folderNameCreate) #MovePDF into folder
        os.chdir(folderNameCreate)
        for j in os.listdir(folderNameCreate):
            print(j)
            if j.endswith(".pdf"): 
                imgName = convert_pdf_2_image(folderNameCreate, j) #call pdf2img function
                print("Running Number: " + str(len(imgName))) 
                printString = ""
                for docName in imgName: # for making the printing better
                    printString += docName + " " 
                print(printString) 
            for q in range(len(imgName)):
                text = convert_img_2_text(folderNameCreate + i[:-4] + "_" + str(q+1) + ".jpg")
                textList.append(text)     #Convert to text and append all the text to list
                numberPagesList.append(len(imgName))   
    else:
        os.chdir(folder_pdf) #delete any other file that exists
        os.remove(i)

/Users/kunal/Documents/VdartResumeProject/runningDoc/Fold_Document_115/
Document_115.pdf
Running Number: 2
Document_115_1.jpg Document_115_2.jpg 


## Convert to Excel

In [773]:
df = pd.DataFrame()  # creates a tabel with all the text and pages
df['Text Extracted'] = textList
df['DocumentNum'] = numberPagesList

In [774]:
os.chdir(excelFilesPath)

In [775]:
EXCELFILENAME = documentRun[:-4] + "_text.xlsx"
df.to_excel(EXCELFILENAME, index = False) # save the text to the excel file folder

## Read Excel

In [776]:
EXCELFILENAME = documentRun[:-4] + "_text.xlsx" # read the same file

In [777]:
os.chdir(excelFilesPath) # change working path to excel files folder

In [778]:
text = pd.ExcelFile(EXCELFILENAME) 
dftext = text.parse("Sheet1") # read the file to see what was saved

In [779]:
dftext

,Text Extracted,DocumentNum
0,"SCOTT JOHNSON\nCumming, GA ·(937) 903-6163\n• ...",2
1,Worked with various internal teams to help sup...,2


In [780]:
text_list = dftext["Text Extracted"].tolist() # converts both columns to lists 
pagesList = dftext["DocumentNum"].tolist()

## Check for Recommendation letters

In [781]:
count=0
numb = 0
recNumList=[]
for i in text_list: # for all the text files scan if the word recommendation is in there
    if "Recommendation" in str(i): 
        recNumList.append(numb)
    numb+=1

In [782]:
recNumList # print all the page numbers with the word in it

[]

# Converts the text files to an array but still split into pages

In [783]:
runningTextFINAL = '\n'.join(textList) # the text is combined

# Create Sentences

In [784]:
print(runningTextFINAL) 

SCOTT JOHNSON
Cumming, GA ·(937) 903-6163
• johnsons71@yahoo.com ·
As an
environment, I am seeking a position with a company that will utilize my
problem-solving and analytical skills as well as my understanding of
advanced accounting concepts.
<perienced and detailed team accountant in a multi-group
EXPERIENCE
03/2014 - 09/2019
11/2009 - 10/2011
SR. OPERATIONAL AND ACCOUNTING CONSULTANT, METLIFE
RETIREMENT & INCOME SOLUTIONS
Cash account reconciliations - reconciled twenty-one cash
accounts across five business groups (Stable Value/Commercial
Paper, Structured Settlements, Income Annuities, Separate
Accounts, Pensions Direct Payment) daily to ensure
incoming/outgoing payments cleared and accounting generated
properly
Suspense/clearing account reconciliations - reconciled twelve
suspense/clearing accounts to ensure accounting generated
properly for premium/benefits accounts
Manual journal entries - cash and suspense reclass, accrual,
adjusting, closing
Lockbox/Wire/ACH Monitoring - Cha

In [785]:
textChoosen = runningTextFINAL # set variables

In [24]:
setnecnceAuto = []
for line in textChoosen.splitlines(): # function that splits all the text by lines, this is all automatic and not accurate
    setnecnceAuto.append(line)

In [25]:
print("CHAR: " + str(len(textChoosen)) + ", LINES: " + str(len(setnecnceAuto))) # print some information about the text

CHAR: 7323, LINES: 139


In [154]:
sentence = ""
sentenceList = []
char = 0
addNumChar = (int(len(textChoosen)/(len(setnecnceAuto))))+1
for j in range(len(setnecnceAuto)): # loops through all the sentences that were created automatically
    #print([str(j) + " " + textChoosen[char:char + addNumChar]])
    char += addNumChar
    addword = input("Current: >>>"+ sentence+ "<<< + >>>"+ setnecnceAuto[j] + "<<<") 
    #asks if they want to add the next sentence to the current running sentence
    if addword == "": #if nothing inputed, it combines the sentence and waits for next value
        sentence += setnecnceAuto[j] + " "
    elif addword == "g": # if "g" then it saves the current running sentence and then reset to 0 and continue from there
        sentenceList.append(sentence[:-1])
        print("Added >>>"+sentence[:-1]+"<<<")
        sentence = ""
        setnecnceAuto[j: j] = ["testfff"]
    elif addword == "fff": # break everything and stop 
        break

Current: >>><<< + >>>NISARGA HASSAN SREEDHAR<<<
Current: >>>NISARGA HASSAN SREEDHAR <<< + >>>San Jose, California |+1 (925) 789-8911| nisarga.nishu20@gmail.com | www.linkedin.com/in/nisarga-sreedhar-39938516b<<<g
Added >>>NISARGA HASSAN SREEDHAR<<<
Current: >>><<< + >>>San Jose, California |+1 (925) 789-8911| nisarga.nishu20@gmail.com | www.linkedin.com/in/nisarga-sreedhar-39938516b<<<
Current: >>>San Jose, California |+1 (925) 789-8911| nisarga.nishu20@gmail.com | www.linkedin.com/in/nisarga-sreedhar-39938516b <<< + >>>EDUCATION:<<<g
Added >>>San Jose, California |+1 (925) 789-8911| nisarga.nishu20@gmail.com | www.linkedin.com/in/nisarga-sreedhar-39938516b<<<
Current: >>><<< + >>>EDUCATION:<<<
Current: >>>EDUCATION: <<< + >>>Master's in Electrical Engineering (Computer Networking), San Jose State University, California, USA.<<<
Current: >>>EDUCATION: Master's in Electrical Engineering (Computer Networking), San Jose State University, California, USA. <<< + >>>Coursework: Internetworki

In [35]:
sentenceList= setnecnceAuto #var names

In [35]:
# pre made sentence list by Kunal
sentenceList = ['Oleg Kotliarsky',
 '(720) 987-8054',
 'olegkot@gmail.com',
 'Profile highlights:',
 'Web Development Engineer',
 'Vast experience designing, programming and leading enterprise web applications development',
 'Proficient in optimal UX solutions',
 'Great experience in developing reusable components to optimize development time and maintenance',
 'Ability to provide technical leadership and clear guidance to development team',
 'High skills to research, evaluate and implement right technical solution for the enterprise application',
 'Technical Knowledge:',
 'JavaScript, TypeScript, ReactJS, Action Script 3.0, Java, PHP',
 'Angular, AngularJS, RXJS, Karma, Java Spring, İBATIS, Apache Struts',
 'Oracle, MongoDB, SQL Server, MYSQL',
 'angular-cli, npm, bower, gulp, GIT',
 'Languages:',
 'Frameworks:',
 'Databases:',
 'Tools:',
 'Professional Experience: Aug. 2017 - April 2020 Senior Web Developer, Comcast, CO',
 'Designed and developed new app features (Columbo - ESL). (Angular6/Angular UI, Remedy, JAVA, Oracle DB):',
 'data driven "case create wizard" with back-end precheck and dynamic restructure of the steps',
 '- case resolve "stepper" with variable number of corresponding relative issues',
 '- reusable components - "keyword" search, attachments list, "add attachments", PDF viewer, util service with multitude of helper functions.',
 '- HTTP request wrappers, HTTP response interceptor for unified error handling',
 'Developed a web app (Columbo - Executive Support Line). (Hybrid AngularJS/Angular UI, Remedy, JAVA, Oracle DB); Maintain and support GIT Hub of the project',
 'Dec. 2013 – June 2017 Senior UI Developer / Team Lead / Scrum Master, DN2K, CO',
 'Developed customers, search, navigation modules for "MyDairyCentral" web app portal, sensors tiles carousel, responsive design, etc. (Angular4, angular-cli, Jasmine/Karma)',
 'Developed sensors tiles carousel, set karma unit tests framework for "MyGrowCentral" web app portal (AngularJS, Node, responsive design, JHipster, Jasmine/Karma)',
 'Developed different features of the "MyAGCentral" web app portal, including navigation tree, work orders flow, etc. (AngularJS, Node, Mongo) - client Sagelnsights: https://www.sageinsights.com/',
 'Transitioned from Backbone to Angular framework (team effort) of the "MyAGCentral" web app portal',
 'October 2011 - Dec. 2013 Web Developer Expert, Amdocs Inc., CO',
 'Developed "Order Entry" web application for entering order to the Amdocs Enterprise order management system (HTML5/CSS3, JavaScript/jQuery, AJAX/DWR, JSP/Java 7, Oracle, Java Spring, iBatis)',
 'Developed e-signature web app using previously developed JavaScript/jQuery/JSP/Java6/Spring2 framework for sending client\'s contract to "on-the-fly" signature creation (integration with docusign.com service).',
 'Developed part of the real time payment integration flow utilizing Amdocs EAI framework called JESI. (Java, JSP, SOAP, Oracle, JavaScript, jQuery)',
 'Developed Flex "Executive Advisor" report tool for serving different types of client statistics presented in a rich graphic interactive way (Flex 4.6, Blaze DS, Java, Oracle, Action Script 3)',
 'October 2010 – Oct. 2011 Web Developer Contractor, Rose International (for Amdocs Inc.), CO',
 'Developed iLink Mobile app (running on iPad Safari – used home developed framework) for sales representatives [client: DexOne]. (JavaScript, jQuery, jQTouch, AJAX, HTML5/webkit, Java, Spring, iBatis, Oracle, JSP);',
 'Developed an iPad web application framework for rapid development of iOS apps that look like',
 'March 2009 - October 2010 GSET Engineer, Wall Street on Demand (now Markit on Demand), Boulder, CO',
 'Developed Entitlements management intranet tool [client: Goldman Sachs]. (Java, Struts, Sybase, JSP, JavaScript, jQuery, AJAX);',
 'March 2008 – March 2009 Team Lead Developer, Wall Street on Demand (now Markit on Demand), Boulder, CO',
 'Leaded team of web developers, working on line of Stocks Research Websites [client: Schwab Institutional] (ASP, JavaScript, AJAX);',
 'August 2005 – March 2008 Senior Web Developer, Wall Street on Demand (now Markit on Demand), Boulder, CO',
 'Developed Web site architecture and determine software requirements.',
 'Created and optimized content for the Web site, including planning, design, integration and testing of Web-site related code.',
 'Planned and designed new featured web sites according to client requirements. Close interaction with graphic designers, project managers and QA members of our group;',
 'Developed Real Time DB driven web sites with Stocks market content, including price quotes graphics, charts, news, alerts etc. Schwab group projects (ASP, JScript, JavaScript, AJAX);',
 'April 2004 – August 2005 Freelancer Web Developer, Denver, CO',
 'Developed full code circle from templates to launch (PHP/MySql, JavaScript, HTML, CSS);',
 'Created Graphics: logos, bullets, complete design (Photoshop, Flash MX);',
 'Promoted web sites in search engines positioning (1st page positions in Google, Yahoo on several key words).',
 'April 2003 – April 2004',
 'Web developer, Gteko (purchased by Microsoft), Raanana, Israel',
 '- Developed JavaScript/VBScript active client side (ActiveX event\'s handling flow: version checking, upgrading, downloading and installation) of different "e-support" accounts (HP, Canon, AOL, Dell, NEC, Lenovo, etc)',
 'Developed JavaScript classes reflecting graphic presentation of ActiveX control downloading and installation processes;',
 'Developed a full JavaScript based interface for ActiveX control data processing - JavaScript/DOM/DHTML based model for PC scanned data show. Was a leading developer for GTWebCheck product part called "Upgrade Advisor" or "Summary Report".',
 'June 2002 – April 2003 Freelancer Web developer, Tel-Aviv , Israel',
 'Complete web sites production (Programming development, PHP/MYSQL/JavaScript/HTML/CSS index + forum (OOP);',
 "Created graphic design according to Client's requirements (logo, bullets, layout – Photoshop, Flash); Made domain name registration; Assisted in identity development, marketing, online promotion and launch;",
 'Maintained web mastering; Promoted web sites for search engines positioning August 1999 - May 2002',
 'Web Developer, Snapshield Ltd, Tel Aviv, Israel',
 "Created Web design (Photoshop, Flash) for Web based application for remote data management for tens of thousands of clients of the leading Snapshield's Telecom Encryption Service (SNAP);",
 'Programmed part of the SNAP application (Customer Care) using ASP, JavaScript, VBScript, CSS, IIS 4;',
 'Set required configuration for SSL on MS IIS4.',
 'Created web based client-side application "Snapshield\'s Security Algorithm Benchmark" for dynamic online calculating and show for different TI DSP platforms and Snapshield algorithms. (ASP, CSS, DHTML)',
 'Created Flash Animated Company Business and Technical Presentations (online, cds) in Macromedia',
 'Flash 5; Integrated video streaming to companies web site (JavaScript, DHTML)',
 'Created graphic and technical design, developed, published and web mastered three generations of the\ncompany\'s web site. (HTML/DHTML, JavaScript, CSS, Macromedia Flash 5, Adobe Photoshop 5.5;',
 'Assisted in new branding process (migrating from Microlink to Snapshield)',
 'Education:',
 'BS and MS in Mathematics and Mechanics',
 'St. Petersburg State University',
 'Courses:',
 '"Oracle Certified Associate", iTerra Consulting, Ic., Denver, USA.',
 '"Design for Multimedia", ORT Syngalovsky College, Tel Aviv. (800 hours)',
 '"JavaScript - DHTML – DOM", Sela group, Tel Aviv (30 hours)',
 '"OOD/OOP for C++ and Java", "Network TCP/IP", Tel-Ran, Rishon-Lezion (1000 hours)']

# Automatic Entity Creation

In [1]:
#https://cloud.google.com/natural-language/docs/basics#entity_analysis
def extractMajorEntities(text_content, salienceScoreThres):
    keyDIR = nlpAutoAPIPath # JSON path with the crediatials 
    credentials = service_account.Credentials.from_service_account_file(keyDIR) # same as before api credentials
    client = language_v1.LanguageServiceClient(credentials=credentials)  # language client 
    type_ = enums.Document.Type.PLAIN_TEXT 
    language = "en" # english text
    document = {"content": text_content, "type": type_, "language": language} #formatting the design of api input -- leave it
    encoding_type = enums.EncodingType.UTF8 # reading the document -- must have 
    response = client.analyze_entities(document, encoding_type=encoding_type) # actual run of the api
    majorValues = []
    for entity in response.entities:  #reading the JSON that was returned by analyze entities
        if entity.salience > salienceScoreThres: #salience score is the importance of the entity compared to the full document
            majorValues.append([entity.name,entity.salience]) # if it is greater than the threshold then save it and return it
    return  majorValues
def extractEntities(text_content):
    keyDIR = nlpAutoAPIPath# JSON path with the crediatials 
    credentials = service_account.Credentials.from_service_account_file(keyDIR) # same as before api credentials
    client = language_v1.LanguageServiceClient(credentials=credentials) # language client
    type_ = enums.Document.Type.PLAIN_TEXT
    language = "en"# english text
    document = {"content": text_content, "type": type_, "language": language}#formatting the design of api input -- leave it
    encoding_type = enums.EncodingType.UTF8# reading the document -- must have 
    response = client.analyze_entities(document, encoding_type=encoding_type) # actual run of the api
    nonuseEntities = ["OTHER", "NUMBER"] # attributes of the entities that dont matter and are of no use
    allEntitiesExtracted = []
    for entity in response.entities: # loop through json and see all entities
        currentRunningEntity = []
        #print(entity.name)
        #print(enums.Entity.Type(entity.type).name)
        if enums.Entity.Type(entity.type).name not in nonuseEntities: # check if meta data from entity is important
            #print("Detected " + entity.name + " as " + enums.Entity.Type(entity.type).name)
            currentRunningEntity.append(entity.name)
            currentRunningEntity.append(enums.Entity.Type(entity.type).name)
            allEntitiesExtracted.append(currentRunningEntity) # append it to list 
        else: 
            for mention in entity.mentions:
                if enums.EntityMention.Type(mention.type).name == "PROPER":
                    #print("Detected " + mention.text.content + " as " + enums.EntityMention.Type(mention.type).name)
                    currentRunningEntity.append(mention.text.content)
                    currentRunningEntity.append(enums.EntityMention.Type(mention.type).name)
                    allEntitiesExtracted.append(currentRunningEntity) # another check if it entity is important 
                    # must see format or return entity to see how this is working 
        #allEntitiesExtracted.append(currentRunningEntity) if currentRunningEntity != [] else print("-")
        #allEntitiesExtracted.append(currentRunningEntity)   
    finalArray = []
    runningEnitityCount = 0
    positionValuesEnitiesList = []
    for i in range(len(allEntitiesExtracted)):
        arrayrun = []
        arrayrun.append(allEntitiesExtracted[i][0])
        arrayrun.append(allEntitiesExtracted[i][1])
        arrayrun.append(text_content.find(allEntitiesExtracted[i][0]))
        positionValuesEnitiesList.append(arrayrun)
    positionValuesEnitiesList = sorted(positionValuesEnitiesList, key=lambda x: x[2]) # sort everything by what is first 
    #print(positionValuesEnitiesList)
    # some of the entities should be combined together becuase they are 2 words and they both fit the position of what the 
    # entity is so, the following code checks the distance between the 2 words and see if they should be together and if so 
    # it return with the entire phrase as a entity 
    runningPositionValuesCount = 0 
    for numb in range(len(positionValuesEnitiesList)-1):
        distancebetween = 0
        lenWord = len(positionValuesEnitiesList[numb][0])
        wordPosition = positionValuesEnitiesList[numb][2]
        distancebetween =((positionValuesEnitiesList[numb+1][2] - (wordPosition + lenWord)))
        sameElement = True if positionValuesEnitiesList[numb][1] == positionValuesEnitiesList[numb+1][1] else False
        #print(sameElement)
        if distancebetween < 2.5 and sameElement: # 2.5 is distance
            #print(positionValuesEnitiesList[numb])
            returnArray = [text_content[positionValuesEnitiesList[numb][2]:positionValuesEnitiesList[numb+1][2]+len(positionValuesEnitiesList[numb+1][1])]]
            returnArray.append(positionValuesEnitiesList[numb][1])
            finalArray.append(returnArray)
        else: 
            finalArray.append([positionValuesEnitiesList[numb][0], positionValuesEnitiesList[numb][1]])
    #print(distancebetween)
    #if distancebetween/len(positionValuesEnitiesList)-1 < 2.5:
        #returnArray = [text_content[positionValuesEnitiesList[0][2]:positionValuesEnitiesList[-1][2]+len(positionValuesEnitiesList[-1][1])]]
        #returnArray.append(positionValuesEnitiesList[0][1])
        #print(returnArray)
    return finalArray

In [37]:
#run everything above 
sentenceArrayWithEntities = []
for i in sentenceList:
    temparray = []
    temparray.append(i)
    entityArray = extractEntities(i)
    for j in entityArray:
        temparray.extend(j)
    sentenceArrayWithEntities.append(temparray)

In [38]:
importantEntitiesArray = extractMajorEntities(runningTextFINAL, 0.05)
print(importantEntitiesArray)

[['team', 0.053995128720998764]]


# Check what document is about

In [39]:
def sample_classify_text(text_content):
    keyDIR = nlpAutoAPIPath # same as before api credentials
    credentials = service_account.Credentials.from_service_account_file(keyDIR) # same as before
    client = language_v1.LanguageServiceClient(credentials=credentials)
    type_ = enums.Document.Type.PLAIN_TEXT
    language = "en"
    document = {"content": text_content, "type": type_, "language": language}
    response = client.classify_text(document)
    # Loop through classified categories returned from the API
    categoryList = []
    # this api returns what category the entire document is about and see what it really means
    for category in response.categories:
        # Get the name of the category representing the document.
        # See the predefined taxonomy of categories:
        # https://cloud.google.com/natural-language/docs/categories
        # print(u"Category name: {}".format(category.name))
        categoryList.append([category.name, category.confidence])
    return categoryList

In [51]:
allPossibleCategory = sample_classify_text(runningTextFINAL)
for category in allPossibleCategory:
    print(("The Document is about: " + '\033[1m' + category[0] + '\033[0m' + 
           " \t with a " + '\033[1m' + "{:.1f}"+ '\033[0m' + " accuracy.\n").format(category[1]*100))
#print(allPossibleCategory)
# print in a bold way

The Document is about: /Finance/Accounting & Auditing 	 with a 95.0 accuracy.

The Document is about: /Business & Industrial 	 with a 55.0 accuracy.



# Save Sentences and Entities|

In [41]:
os.chdir(excelFilesPath)
# saved what was created in the excel folder place but with the following name

In [42]:
df = pd.DataFrame(sentenceArrayWithEntities)
EXCELFILENAMESent = documentRun[:-4] + "_sentencesNew.xlsx"
df.to_excel(EXCELFILENAMESent)

In [43]:
EXCELFILENAMESent
# name of the file with the sentences

'Document_118_sentencesNew.xlsx'

# --- HUMAN MUST MANUALLY INPUT ENTITIES ---
**CHECK THIS LINK FOR MORE INSTRUCTIONS**

https://docs.google.com/document/d/1BdbWL7ePGPYiSVa07g8Erknwm_Zmi2JUwp588vJdUug/edit?usp=sharing

# Convert Array to JSON

In [786]:
EXCELFILENAMESent = documentRun[:-4] + "_sentencesNew.xlsx"
# reads the excel name

In [787]:
EXCELFILENAMESent = '/Users/kunal/Documents/VdartResumeProject/ExcelFiles/OtherRun/Document_115_sentencesNew.xlsx'

In [788]:
#EXCELFILENAMESent = 'Document_119_sentencesNew.xlsx'
# if you want to call a sperate name

## Extract Excel and Turn into a list

In [789]:
os.chdir(excelFilesPath)

In [790]:
text = pd.ExcelFile(EXCELFILENAMESent)
totalTextRunning = runningTextFINAL
attributes = text.parse("Sheet1") 
# read excel

In [791]:
attributes
# this is a table of all of the entities. 
# IMPORTANT -- the name of the first row with the text must be named "Text"
# could be every how long columns

,Text,1,2,3,4,5,6
0,SCOTT JOHNSON,Name,All,NaN,NaN,NaN,NaN
1,"Cumming, GA ·",Address,All,NaN,NaN,NaN,NaN
2,(937) 903-6163,Phone,All,NaN,NaN,NaN,NaN
3,johnsons71@yahoo.com ·,Email,All,NaN,NaN,NaN,NaN
4,As an,None,NaN,NaN,NaN,NaN,NaN
5,"environment, I am seeking a position with a co...",None,NaN,NaN,NaN,NaN,NaN
6,problem-solving and analytical skills as well ...,None,NaN,NaN,NaN,NaN,NaN
7,advanced accounting concepts.,None,NaN,NaN,NaN,NaN,NaN
8,<perienced and detailed team accountant in a m...,None,NaN,NaN,NaN,NaN,NaN
9,EXPERIENCE,None,NaN,NaN,NaN,NaN,NaN


In [792]:
RowList = attributes.values.tolist()
attributeList = [[i for i in row if isinstance(i,str)] for row in RowList]
#Converts to a 2d array 

## Create Final List

In [793]:
def sentence_2_word(sentence):
    # this functuion takes a sentence and splits it into each word and returns a list of all the words
    word = ""
    listofWords = []
    for i in sentence:
        if not (i == " " or i == "\n"):
            word += i 
        else:
            listofWords.append(word)
            word = ""
    if len(listofWords) == 0:
        listofWords.append(sentence)
    return listofWords

In [794]:
def find_all_indexes(input_str, search_str):
    # find all the positions of where a text is in a string
    # not used so dont worry
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1[0]

In [795]:
def checkNotWorkingValues(textFindPosition, totalTextRunning):
    # if the find function cant find the text in the total text then this function splits the search text into each word 
    # find its value out of the entire text and if they are close enough then then it returns the position of the starting 
    # word of the search text but if it can't get a definitie answer then it returns -1 
    words = textFindPosition.split()
    wordPositions = []
    for singleWord in words:
        wordPositions.append(totalTextRunning.find(singleWord))
    positionsCount = 0
    numCorrect = 0
    for num in range(len(wordPositions)-1):
        if wordPositions[num]+len(words[num])+1 == wordPositions[num+1]:
            numCorrect+=1
        positionsCount+=1
    if numCorrect == len(words)-1:
        return wordPositions[0]
    else:
        return -1

In [796]:
def checkNotWorkingValues(textFindPosition, totalTextRunning):
    words = textFindPosition.split()
    if len(words) >= 7:
        wordListNew = []
        for number in range(1, len(words), 2):
            wordListNew.append(words[number-1] + " " + words[number])
        if not len(words) % 2 == 0: 
            wordListNew.append(words[-1])
        #print(wordListNew)
    words = wordListNew
    wordPositions = []
    numberstarting = 0  
    for singleWord in words:
        #print(totalTextRunning[numberstarting:].find(singleWord))
        wordPositions.append(totalTextRunning[numberstarting:].find(singleWord)+numberstarting)
        if not totalTextRunning[numberstarting:].find(singleWord) == -1:
            numberstarting = totalTextRunning[numberstarting:].find(singleWord) +numberstarting
        #print(numberstarting)    
    #print(wordPositions)
    positionsCount = 0
    numCorrect = 0
    for num in range(len(wordPositions)-1):
        if abs(wordPositions[num]+len(words[num])+1 - wordPositions[num+1]) < 2:
            numCorrect+=1
        #else:
            #print(words[num])
            #print(wordPositions[num])
            #print(len(words[num])+1)
            #print(wordPositions[num]+len(words[num])+1)
            #print(wordPositions[num+1])
            #print("")
        positionsCount+=1
    #print(positionsCount)
    #print(numCorrect)
    if numCorrect/positionsCount > 0.6:
        return wordPositions[0]
    else:
        return -1

In [797]:
possibleEntitiyNames = ["Skill", "Company", "WorkExperience", "Name", "Address", "Phone", "Email", "Date", "Link", 
                        "Education", "Software", "Hardware", "AwardCertification", "None"]
numbAttributeReal = 0
lineCount = 2
entitiesperLine = []
for line in attributeList:
    if len(line) == 2 and not line[1] == "None":
        print("SENTENCE LEFT BLANK \t ERROR IN DATA! Check line \033[1m" + str(lineCount) + 
              "\033[0m. The sentence is \033[1m"+ line[0] + "\033[0m with entity of \033[1m" + 
              line [1] + "\033[0m")
        #break
    if len(line) == 0:
        print("LINE IS BLANK \t ERROR IN DATA!\t Check line \033[1m" + str(lineCount) + "\033[0m.")
    if len(line) > 2 and len(line) % 2 == 0:
        print("ENTITIES NOT COMPLETED ERROR IN DATA!\t Check line \033[1m" + str(lineCount) + "\033[0m.")
        print(line)
    if not line[1] in possibleEntitiyNames:
        print("ENTITIES INVALID ERROR IN DATA!\t Check line \033[1m" + str(lineCount) + "\033[0m. Entity name is: \033[1m" +
             line[1] + "\033[0m.")
        print(line)
    #print(len(line))
    entitiesperLine.append(int((len(line)-1)/2))
    #1if not len(line) == 2:
    lineCount += 1
print("TOTAL OF \033[1m" + str(len(entitiesperLine)) + "\033[0m entities detected")

TOTAL OF 55 entities detected


In [798]:
attributesNumberList = []
for j in attributeList:
    runningAttributeNum = []
    if len(j) == 2:
        continue
    if j[2] == "All": 
        startChar = totalTextRunning.find(j[0].strip())
        if startChar == -1:
            words = j[0].strip().split()
            if len(words) >= 7:
                wordListNew = []
                for number in range(1, len(words), 2):
                    wordListNew.append(words[number-1] + " " + words[number])
                if not len(words) % 2 == 0: 
                    wordListNew.append(words[-1])
                words = wordListNew
            wordPositions = []
            numberstarting = 0  
            for singleWord in words:
                wordPositions.append(totalTextRunning[numberstarting:].find(singleWord)+numberstarting)
                if not totalTextRunning[numberstarting:].find(singleWord) == -1:
                    numberstarting = totalTextRunning[numberstarting:].find(singleWord) +numberstarting
            positionsCount = 0
            numCorrect = 0
            for num in range(len(wordPositions)-1):
                if abs(wordPositions[num]+len(words[num])+1 - wordPositions[num+1]) < 2:
                    numCorrect+=1
                positionsCount+=1
            try: 
                if numCorrect/positionsCount > 0.6:
                    startChar = wordPositions[0]
                else:
                    startChar = -1  
            except ZeroDivisionError:
                startChar = -1 
        if startChar == -1:
            # lower everything and see if it can find it then
            startChar = totalTextRunning.lower().find(j[0].lower().strip())
        endChar = startChar + len(j[0].strip())
        runningAttributeNum.extend([j[1].strip().upper() , startChar, endChar])
        #print(j[1].strip().upper() + "\t" + totalTextRunning[startChar: endChar])
        attributesNumberList.append(runningAttributeNum)
        # print(runningAttributeNum)
    else:
        #print("RUNNING " + str(int((len(j)-1)/2)))
        # im not going to go in detail explaining this but copy this function into a sepreate file and try it out
        # similar to the "all thing" but it works for any number of attributes 
        for w in range(0, int((len(j)-1)), 2):
            runningAttributeNum = []
            if not totalTextRunning.find(j[2+w]) == -1:
                startChar = totalTextRunning.find(j[2+w].strip())
                if startChar == -1:
                    startChar = checkNotWorkingValues(j[2+w].strip(), totalTextRunning)
                if startChar == -1:
                    startChar = totalTextRunning.lower().find(j[2+w].lower().strip())
                endChar = startChar + len(j[2+w])
                runningAttributeNum.extend([j[1+w].upper(), startChar, endChar])
                #print("--"  + j[1].strip().upper() + "\t" + totalTextRunning[startChar: endChar])
            else:
                word = ""
                listofWords = []
                for i in j[2+w]:
                    if not (i == " " or i == "\n"):
                        word += i 
                    else:
                        listofWords.append(word)
                        word = ""
                listofWords.append(word)
                if len(listofWords) == 0:
                    listofWords.append(j[2+w])           
                words = listofWords
                detected = 0
                position = []
                for g in words:
                    if not j[2+w].find(g) == -1:
                        #charTemp = j[2+w].find(g)
                        detected+=1
                        position.append(j[2+w].find(g))
                correct = 0
                allcount = 0
                for thing in range(len(position)-1):
                    if abs(position[thing] + len(words[thing]) - position[thing+1]) < 2:
                        correct+=1
                    allcount+=1
                if correct >= 1:
                    u = words
                    counter2 = 0
                    connectedProbability = []
                    for numb in range(len(u)-1):
                        #print(u[numb])
                        indexArray  = find_all_indexes(totalTextRunning, u[numb])
                        uptadedArray = []
                        for i in indexArray:
                            uptadedArray.append(i+len(u[numb]))
                        indexArrayWord2 = find_all_indexes(totalTextRunning, u[numb+1])
                        #print(indexArrayWord2)
                        for number2 in indexArrayWord2:
                            try:
                                whichNumbClosest = indexArray[min(range(len(indexArray)), key = lambda i: abs(indexArray[i]-number2))]
                            except ValueError:
                                pass
                            #print([whichNumbClosest, number2-1])
                            connectedProbability.append([whichNumbClosest, number2-1])
                            #if number2 > whichNumbClosest and abs((number2-whichNumbClosest)-len(u[numb])) < 3:
                                #connectedProbability.append([whichNumbClosest, number2-1])
                            #else:
                                #connectedProbability.append([whichNumbClosest, number2-1])
                    startChar = connectedProbability[0][0]
                    endChar = startChar + len(j[2+w])
                    runningAttributeNum.extend([words[0].upper(), startChar, endChar])
                else:
                    charTemp = -1 # couldn't finf anything
                    print("NOT DETECTED")
                    endChar = -1
                    startChar = -1
                    runningAttributeNum.extend(["NOTDETECTED", startChar, endChar])
                #print(j[1].strip().upper() + "\t" + totalTextRunning[startChar: endChar])
            attributesNumberList.append(runningAttributeNum)
    if startChar == -1:
        print(j )

NOT DETECTED
['Microsoft Suite', 'Skill', 'Software']
NOT DETECTED
['(Excel/Word/Outlook/OneDrive)', 'Skill', 'MSOffice']


In [799]:
len(attributesNumberList)

47

In [800]:
attributesNumberList

[['NAME', 0, 13],
 ['ADDRESS', 14, 27],
 ['PHONE', 27, 41],
 ['EMAIL', 44, 66],
 ['DATE', 313, 330],
 ['DATE', 331, 348],
 ['WORKEXPERIENCE', 349, 391],
 ['COMPANY', 392, 429],
 ['SKILL', 430, 487],
 ['SKILL', 488, 549],
 ['SKILL', 550, 607],
 ['SKILL', 608, 658],
 ['SKILL', 659, 718],
 ['SKILL', 719, 727],
 ['SKILL', 728, 789],
 ['SKILL', 790, 847],
 ['SKILL', 848, 886],
 ['SKILL', 887, 947],
 ['SKILL', 948, 966],
 ['SKILL', 967, 1028],
 ['SKILL', 1029, 1091],
 ['SKILL', 1092, 1144],
 ['SKILL', 1145, 1208],
 ['SKILL', 1209, 1274],
 ['SKILL', 1275, 1341],
 ['SKILL', 1342, 1389],
 ['SKILL', 1390, 1450],
 ['SKILL', 1451, 1465],
 ['SKILL', 1466, 1520],
 ['SKILL', 1521, 1585],
 ['SKILL', 1586, 1637],
 ['SKILL', 1638, 1703],
 ['SKILL', 1704, 1767],
 ['SKILL', 1768, 1825],
 ['SKILL', 1826, 1880],
 ['SKILL', 1881, 1942],
 ['SKILL', 1943, 1989],
 ['DATE', 2000, 2017],
 ['EDUCATION', 2018, 2058],
 ['NOTDETECTED', -1, -1],
 ['NOTDETECTED', -1, -1],
 ['SKILL', 2112, 2129],
 ['SKILL', 2130, 2153],

In [801]:
len(attributesNumberList)

47

In [802]:
for testing in attributesNumberList:
    # if it can't find it removes it 
    if testing[1] == -1:
        print(testing)
        attributesNumberList.remove(testing)

['NOTDETECTED', -1, -1]


## DataSet to JSON FILE

In [803]:
len(attributesNumberList)

46

In [804]:
attributeCount = len(attributesNumberList)
text = totalTextRunning
attributes = attributesNumberList

In [805]:
text

'SCOTT JOHNSON\nCumming, GA ·(937) 903-6163\n• johnsons71@yahoo.com ·\nAs an\nenvironment, I am seeking a position with a company that will utilize my\nproblem-solving and analytical skills as well as my understanding of\nadvanced accounting concepts.\n<perienced and detailed team accountant in a multi-group\nEXPERIENCE\n03/2014 - 09/2019\n11/2009 - 10/2011\nSR. OPERATIONAL AND ACCOUNTING CONSULTANT, METLIFE\nRETIREMENT & INCOME SOLUTIONS\nCash account reconciliations - reconciled twenty-one cash\naccounts across five business groups (Stable Value/Commercial\nPaper, Structured Settlements, Income Annuities, Separate\nAccounts, Pensions Direct Payment) daily to ensure\nincoming/outgoing payments cleared and accounting generated\nproperly\nSuspense/clearing account reconciliations - reconciled twelve\nsuspense/clearing accounts to ensure accounting generated\nproperly for premium/benefits accounts\nManual journal entries - cash and suspense reclass, accrual,\nadjusting, closing\nLockbox/

In [806]:
def createJSONFILE(attributeCount, text, attributes):
    #the 2 string named "totalString" and "attributeString" are just formated in a way how google likes their input value
    # loops through everything and uses the replace function to replace it with all the attributes
    # you have to test this out yourself to understand how it works
    alphab = list(string.ascii_lowercase)
    for alpa in range(int(attributeCount/26)):
        alphab.extend(list(string.ascii_lowercase))
    totalString = """{"textSnippet":{"content":"textCODE"},"annotations":[attributeCODE]}"""
    attributeString = """{"displayName":"AtribnameCODE","textExtraction":{"textSegment":{"startOffset":"sCharCODE","endOffset":"eCharCODE"}}}"""
    totalString =  totalString.replace("textCODE", text)
    tempStringReplacer = ""
    for i in range(attributeCount):
        if i != attributeCount-1:
            tempStringReplacer+=("atriCODE"+str(i+1)+alphab[i]+",")
        else:
            tempStringReplacer+=("atriCODE"+str(i+1)+alphab[i])
    totalString = totalString.replace("attributeCODE", tempStringReplacer)
    for i in range(attributeCount):
        attributeStringNEW = attributeString
        #print(attributes[i])
        attributeStringNEW = attributeStringNEW.replace("AtribnameCODE", attributes[i][0])
        attributeStringNEW = attributeStringNEW.replace("sCharCODE", str(attributes[i][1])).replace("eCharCODE", str(attributes[i][2]))
        totalString = totalString.replace(("atriCODE"+str(i+1)+alphab[i]), attributeStringNEW)
    return totalString

## Final Run Create JSON

In [807]:
file = createJSONFILE(attributeCount, text, attributes)

In [808]:
attributeCount

46

In [809]:
file

'{"textSnippet":{"content":"SCOTT JOHNSON\nCumming, GA ·(937) 903-6163\n• johnsons71@yahoo.com ·\nAs an\nenvironment, I am seeking a position with a company that will utilize my\nproblem-solving and analytical skills as well as my understanding of\nadvanced accounting concepts.\n<perienced and detailed team accountant in a multi-group\nEXPERIENCE\n03/2014 - 09/2019\n11/2009 - 10/2011\nSR. OPERATIONAL AND ACCOUNTING CONSULTANT, METLIFE\nRETIREMENT & INCOME SOLUTIONS\nCash account reconciliations - reconciled twenty-one cash\naccounts across five business groups (Stable Value/Commercial\nPaper, Structured Settlements, Income Annuities, Separate\nAccounts, Pensions Direct Payment) daily to ensure\nincoming/outgoing payments cleared and accounting generated\nproperly\nSuspense/clearing account reconciliations - reconciled twelve\nsuspense/clearing accounts to ensure accounting generated\nproperly for premium/benefits accounts\nManual journal entries - cash and suspense reclass, accrual,\na

## Save JSONL FILE

In [810]:
os.chdir(jsonFolderPath)
# open a file and save it 
jsonFileName = "Final_New_"+documentRun[9:-4]+".jsonl"
with io.open(jsonFileName, "w", encoding="utf-8") as text_file:
    text_file.write(file)
text_file.close()

In [811]:
print(jsonFileName)

Final_New_115.jsonl


In [812]:
f = open(jsonFileName, "r")
#print(f.read())

Now your done and you have the Json file, give it to kunal and he will put it into GCP platform account

In [813]:
numFilecount = 0
for i in os.listdir(jsonFolderPath):
    if i.startswith("Final_New_"):
        print(i)
        numFilecount+=1
print(numFilecount)

Final_New_100.jsonl
Final_New_105.jsonl
Final_New_106.jsonl
Final_New_108.jsonl
Final_New_109.jsonl
Final_New_112.jsonl
Final_New_113.jsonl
Final_New_115.jsonl
Final_New_119.jsonl
Final_New_122.jsonl
Final_New_125.jsonl
Final_New_139.jsonl
12


In [700]:
#f = open('Final_New_108.jsonl', 'r')
#file_contents = f.read()
#print (file_contents)
#f.close()